Instalasi Library & Import Library

In [ ]:
!pip install google-play-scraper pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from google_play_scraper import reviews, Sort
from tqdm import tqdm

ID Aplikasi Google Play Store

In [ ]:
apps = {
    "Ruangguru": "com.ruangguru.livestudents",
    "Pahamify": "com.pahamify.android",
    "Quipper": "com.quipper.school.assignment"
}

Fungsi Scraping Review

In [ ]:
def scrape_reviews(app_name, app_id, n_reviews=2000):
    all_reviews = []
    continuation_token = None

    with tqdm(total=n_reviews, desc=f"Scraping {app_name}") as pbar:
        while len(all_reviews) < n_reviews:
            result, continuation_token = reviews(
                app_id,
                lang='id',
                country='id',
                sort=Sort.NEWEST,
                count=2000,
                continuation_token=continuation_token
            )

            if not result:
                break

            for r in result:
                all_reviews.append({
                    'app': app_name,
                    'review': r['content'],
                    'rating': r['score'],
                    'date': r['at']
                })
                pbar.update(1)

                if len(all_reviews) >= n_reviews:
                    break

            if continuation_token is None:
                break

    return pd.DataFrame(all_reviews)

Jalankan Scraping (Total 6000 Data)

In [ ]:
all_data = []

for app_name, app_id in apps.items():
    df_app = scrape_reviews(app_name, app_id, n_reviews=2000)
    all_data.append(df_app)

df_reviews = pd.concat(all_data, ignore_index=True)

Scraping Quipper: 100%|██████████| 2000/2000 [00:01<00:00, 1984.81it/s]


Cek Total Data Keseluruhan

In [ ]:
print("Total seluruh data:", len(df_reviews))

Total seluruh data: 6000


Cek Jumlah Data Per-Aplikasi

In [ ]:
print("\nJumlah data per aplikasi:")
print(df_reviews['app'].value_counts())


Jumlah data per aplikasi:
app
Ruangguru    2000
Pahamify     2000
Quipper      2000
Name: count, dtype: int64


Atribut Data

In [ ]:
for app in df_reviews['app'].unique():
    print(f"\n===== Contoh data {app} =====")
    display(df_reviews[df_reviews['app'] == app].head())


===== Contoh data Ruangguru =====


,app,review,rating,date
0,Ruangguru,kenapa ya selalu update setiap mau masuk ke ha...,1,2026-01-11 13:49:55
1,Ruangguru,bagussd,5,2026-01-11 13:22:52
2,Ruangguru,"banyak bug!!!,kalau buka aplikasi sering terke...",1,2026-01-11 11:28:11
3,Ruangguru,wahhh sangat 🅑🅐🅖🅤🅢,4,2026-01-11 10:50:11
4,Ruangguru,good,5,2026-01-11 08:13:04



===== Contoh data Pahamify =====


,app,review,rating,date
2000,Pahamify,"Aga kecewa si, pdhl udah berlangganan. Mau pak...",2,2026-01-11 14:09:02
2001,Pahamify,"Pembayaran dialihkan ke website, tidak didalam...",1,2026-01-11 03:02:30
2002,Pahamify,"server lelet, suka ngebug, dn paket tdk bisa d...",1,2026-01-10 15:11:08
2003,Pahamify,"Dari segi kualitas dah lumayan, tapi masih per...",4,2026-01-08 09:42:02
2004,Pahamify,Sejauh ini berlangganan cukup baguss tapi masa...,4,2026-01-06 08:54:10



===== Contoh data Quipper =====


,app,review,rating,date
4000,Quipper,"gak tau kenapa, tapi search buat materi di hp ...",1,2026-01-09 11:34:53
4001,Quipper,"sistem ujian ga aman, banyak orang melakukan k...",1,2025-11-29 07:43:28
4002,Quipper,berguna😆,5,2025-11-23 07:14:00
4003,Quipper,"gak jelas, gak seru, sok asik nge gantiin kertas",1,2025-11-21 01:37:46
4004,Quipper,tiba tiba akun saya hilang,1,2025-11-19 01:33:40


In [ ]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   app     6000 non-null   object        
 1   review  6000 non-null   object        
 2   rating  6000 non-null   int64         
 3   date    6000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 187.6+ KB


Waktu Pengambilan Data

In [ ]:
print("Rentang waktu pengambilan data:")
print(f"Tanggal awal: {df_reviews['date'].min()}")
print(f"Tanggal akhir: {df_reviews['date'].max()}")

Rentang waktu pengambilan data:
Tanggal awal: 2020-11-20 14:36:03
Tanggal akhir: 2026-01-11 14:09:02


In [ ]:
print("Rentang waktu pengambilan data per aplikasi:")
for app_name in df_reviews['app'].unique():
    df_app_filtered = df_reviews[df_reviews['app'] == app_name]
    min_date = df_app_filtered['date'].min()
    max_date = df_app_filtered['date'].max()
    print(f"  {app_name}: Tanggal awal: {min_date}, Tanggal akhir: {max_date}")

Rentang waktu pengambilan data per aplikasi:
  Ruangguru: Tanggal awal: 2025-06-27 10:21:18, Tanggal akhir: 2026-01-11 13:49:55
  Pahamify: Tanggal awal: 2021-09-08 08:19:00, Tanggal akhir: 2026-01-11 14:09:02
  Quipper: Tanggal awal: 2020-11-20 14:36:03, Tanggal akhir: 2026-01-09 11:34:53


Cek Apakah Ada Review Kosong

In [ ]:
print("Review kosong:", df_reviews['review'].isna().sum())
print("Review string kosong:", (df_reviews['review'].str.strip() == '').sum())

Review kosong: 0
Review string kosong: 0


In [ ]:
df_reviews.to_csv("/content/drive/MyDrive/Mini_Project/review_aplikasi_belajar.csv", index=False)
print("Scraping selesai!")
print(df_reviews['app'].value_counts())

Scraping selesai!
app
Ruangguru    2000
Pahamify     2000
Quipper      2000
Name: count, dtype: int64
